%pip install unstructured

%pip install python-docx

%pip install pypdfium2

%pip install -qU langchain-text-splitters

%pip install -U langgraph langsmith langchain_anthropic

%pip install paramiko

%pip install torch ChatTTS

%pip install nvidia-riva-client

%git clone https://github.com/nvidia-riva/python-clients.git


In [108]:
import os
NVIDIA_API_KEY = "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"
os.environ["NVIDIA_API_KEY"] = NVIDIA_API_KEY

AUTODL_HOST="connect.seetacloud.com"
AUTODL_PORT=0000
AUTODL_USER=""
AUTODL_PASSWORD=""

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "lsv21bc7ffc"
os.environ["LANGCHAIN_PROJECT"] = "beauty_assistant"

In [109]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_nvidia_ai_endpoints import ChatNVIDIA

embedder = NVIDIAEmbeddings(model="NV-Embed-QA")#ai-embed-qa-4
llm_deepseek_r1 = ChatNVIDIA(
      model="deepseek-ai/deepseek-r1",
      temperature=0.6,
      top_p=0.7,
      max_tokens=4096,
    )

In [110]:
from utility import print_with_time

初始向量数据库，只需执行一次

In [111]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from utility import LoadPDF

def init_vector_store():
    have_store=False
    docs=LoadPDF("./kdoc")
    for doc in docs:
        #print_with_time(doc)
        text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size=100,
            chunk_overlap=20,
            length_function=len,
            is_separator_regex=False,
        )
        texts = text_splitter.split_documents(doc)
        print_with_time(texts[0])
        if not have_store:
            store = FAISS.from_documents(texts, embedder)
            have_store=True
        else:
            tmp_store=FAISS.from_documents(texts, embedder)
            store.merge_from(tmp_store)
    store.save_local('./embedding')
    
# 初始向量数据库，只需执行一次
# init_vector_store()

In [112]:
import subprocess
from entity import State

def asr(state:State):
    audio_file=state["question_audio"]
    cmd=f"""python python-clients/scripts/asr/transcribe_file_offline.py --server grpc.nvcf.nvidia.com:443 --use-ssl  --metadata function-id "b702f636-f60c-4a3d-a6f4-f3568c13bd7d" --metadata "authorization" "Bearer {NVIDIA_API_KEY}"  --language-code en --input-file {audio_file}"""
    print_with_time(cmd)
    process  = subprocess.Popen(cmd, stdout=subprocess.PIPE, text=True)
    output, errors = process.communicate()
    # print( output, errors )
    arr=output.split("Final transcript: ")
    if(len(arr)>=2):
        return {"question":arr[1],"messages": [{"role": "user", "content": arr[1]}]}
    else:
        print_with_time( output )
        raise ValueError("asr出现错误")
    
# s=State(question_audio="./asr_example_zh.wav")
# r=asr(s)
# print(r)

In [113]:
import uuid
import re
import torch
import torchaudio
import ChatTTS
from IPython.display import Audio
from utility import is_file_empty

 
def is_chinese(s):
    pattern = re.compile(r'[\u4e00-\u9fff\u3400-\u4DBF\u20000-\u2A6DF]+')
    return bool(pattern.match(s))

def translate(str):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """你是一个中英文翻译专家，将用户输入的中文翻译成英文。直接翻译，不要解析内容。请简洁回答，不重复用户问题，并控制在100字以内。
                """,
            ),
            ("user", f"{str}"),
        ]
    )
    
    chain = (
        prompt
        | llm_deepseek_r1
        | StrOutputParser()
    )
    result=chain.invoke({})
    # print_with_time(result)
    arr=result.split("</think>")
    # print_with_time(arr[-1])
    if len(arr[-1])<=0:
        print_with_time(result)
        raise ValueError("中文翻译英文出现错误")
    return arr[-1]
    
# print(translate("春景短詩，捕捉梅香鳥驚、風過花落的瞬間，末句化用「花開花落知多少」感時傷逝，意象靈動，餘韻悠長。"))

def tts(state:State):
    answer=state["answer"].replace("'","").replace('"','')
    if is_chinese(answer):
        answer=translate(answer) #接口不支持中文j
    audio_path="./audio"
    if not os.path.exists(audio_path):
        os.makedirs(audio_path)
    output_audio=f"{audio_path}/{uuid.uuid4()}.wav"
    cmd=f"""python python-clients/scripts/tts/talk.py  --server grpc.nvcf.nvidia.com:443 --use-ssl --metadata function-id "5e607c81-7aa6-44ce-a11d-9e08f0a3fe49"  --metadata authorization "Bearer {NVIDIA_API_KEY}" --text "{answer}" --voice "English-US-RadTTS.Female-1" --output {output_audio}"""
    print_with_time(cmd)
    process  = subprocess.Popen(cmd, stdout=subprocess.PIPE, text=True)
    output, errors = process.communicate()
    if is_file_empty(output_audio):
        print_with_time( output, errors )
        raise ValueError("tts出现错误")
    return  {"answer_audio":output_audio}

# s=State(answer="春景短詩，捕捉梅香鳥驚、風過花落的瞬間，末句化用「花開花落知多少」感時傷逝，意象靈動，餘韻悠長。")
# r=tts(s)
# print(r)

def chattts(state:State):
    answer=state["answer"]

    audio_path="./audio"
    if not os.path.exists(audio_path):
        os.makedirs(audio_path)
    output_audio=f"{audio_path}/{uuid.uuid4()}.wav"

    # 初始化ChatTTS
    chat = ChatTTS.Chat()
    chat.load()
    # 定义要转换为语音的文本
    texts = [answer]
    # 生成语音
    wavs = chat.infer(texts, use_decoder=True)
    torchaudio.save(output_audio, torch.from_numpy(wavs[0]), 24000)
    # 播放生成的音频
    Audio(wavs[0], rate=24_000, autoplay=True)
    if is_file_empty(output_audio):
        print_with_time( output, errors )
        raise ValueError("tts出现错误")
    return  {"answer_audio":output_audio}

# s=State(answer="春景短詩，捕捉梅香鳥驚、風過花落的瞬間，末句化用「花開花落知多少」感時傷逝，意象靈動，餘韻悠長。")
# r=chattts(s)
# print(r)

In [114]:
import paramiko
import traceback
from entity import State

def audio2face(state:State):
    try:
        answer_audio=state["answer_audio"]
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(hostname=AUTODL_HOST, port=AUTODL_PORT, username=AUTODL_USER, password=AUTODL_PASSWORD)
    
        file_name = os.path.basename(answer_audio)
        remote_audio_file=f"/root/Sonic/examples/wav/{file_name}"
        # 创建SFTP客户端
        sftp = paramiko.SFTPClient.from_transport(ssh.get_transport())
        # 上传文件 /root/autodl-tmp/
        sftp.put(answer_audio, remote_audio_file)
        # 关闭SFTP客户端
        sftp.close()

        file_name=f"{uuid.uuid4()}.mp4"
        remote_video_file=f"/root/Sonic/examples/wav/{file_name}"
        cmd=f"/root/miniconda3/bin/python /root/Sonic/demo.py '/root/Sonic/examples/image/hair.png' '{remote_audio_file}' '{remote_video_file}'"
        print_with_time(cmd)
        # 执行远程命令
        # stdin, stdout, stderr = ssh.exec_command("sudo su")
        # stdin.write(f"{AUTODL_PASSWORD}\n")
        # stdin.flush()
        stdin, stdout, stderr = ssh.exec_command(cmd)
        # 打印命令输出
        print("stdout",stdout.read().decode())
        print("stderr",stderr.read().decode())
        
        video_path="./video"
        if not os.path.exists(video_path):
            os.makedirs(video_path)
        local_video_file=f"{video_path}/{file_name}"
         # 创建SFTP客户端
        sftp = paramiko.SFTPClient.from_transport(ssh.get_transport())
        # 下载文件 /root/autodl-tmp/
        sftp.get(remote_video_file, local_video_file)
        # 关闭SFTP客户端
        sftp.close()
        
        ssh.close()

        if is_file_empty(local_video_file):
            print("stdout",stdout.read().decode())
            print("stderr",stderr.read().decode())
            raise ValueError("audio2face出现错误")
        return  {"answer_video":local_video_file}
    except Exception as e:
        error_msg = traceback.format_exc()
        print_with_time(error_msg)
        raise ValueError("audio2face出现错误")
        
    return  {"answer_video":""}
    

# s=State(answer_audio="./audio/c07eaa93-66ae-41df-8c82-64a6dd6be988.wav")
# r=audio2face(s)
# print(r)

In [115]:
# Router
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from utility import PPrint
from entity import RouteQuery,State

def route_question(state:State):
    """
    Route question to direct answer or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    print_with_time("route_question")
    
    system = """You are an expert at routing a user question to a vectorstore or web search.
    The vectorstore contains documents related to internal company information, IT technical knowledge, and adversarial attacks.
    Use the vectorstore for questions on these topics. Otherwise, use direct_answer.
    以 JSON 的形式输出，输出的 JSON 需遵守以下的格式：

    {{
      "datasource": <数据来源，值为 vectorstore 或者 direct_answer>
    }}"""
    route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "{question}"),
        ]
    )
    # print(route_prompt)
    
    structure_parser = PydanticOutputParser(pydantic_object=RouteQuery)
    # print(structure_parser.get_format_instructions())

    chain = (
        route_prompt
        | llm_deepseek_r1 
        # | PPrint()
        | structure_parser
        # | StrOutputParser()
    )
    question = state["question"]
    source =chain.invoke({"question": question})
    if source.datasource == "vectorstore":
        return "vectorstore"
    else:
        return "direct_answer"

# s=State(question="手语识别(SLR)技术是什么")
# print(route_question(s))

In [116]:
# 
from langchain.vectorstores import FAISS

def retrieve(state):
    print_with_time("retrieve")
    
    question = state["question"]

    store = FAISS.load_local("./embedding", embedder,allow_dangerous_deserialization=True)
    retriever = store.as_retriever(search_kwargs={"k": 3})
    documents  = retriever.invoke(question)
    return {"documents": documents }

# s=State(question=" 不a见")
# print(retrieve(s))

In [117]:
# Retrieval Grader
from entity import GradeDocuments
from entity import State

def grade_documents(state):
    print_with_time("grade_documents")
    
    question=state["question"]
    documents=state["documents"]
    
    # Prompt
    system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
        If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
        It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
        Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question.
        以 JSON 的形式输出，输出的 JSON 需遵守以下的格式：

        {{
          "binary_score": <相关度，值为 yes 或者 no>
        }}"""
    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
        ]
    )

    structure_parser = PydanticOutputParser(pydantic_object=GradeDocuments)
    # print(structure_parser.get_format_instructions())

    chain = (
        grade_prompt
        | llm_deepseek_r1 
        # | PPrint()
        | structure_parser
        # | StrOutputParser()
    )
    
    filtered_docs=[]
    for d in documents:
        score=chain.invoke({"question": question, "document": d.page_content})
        # print(d,score)
        grade=score.binary_score
        if grade=="yes":
            filtered_docs.append(d)
            
        # break
            
    return {"documents": filtered_docs }

# s=State(question="手语识别(SLR)技术是什么")
# docs=retrieve(s)
# s=State(question="手语识别(SLR)技术是什么",documents= docs["documents"])
# print(grade_documents(s))

In [118]:
def decide_to_generate(state):
    print_with_time("decide_to_generate")
    
    documents=state["documents"]
    if(len(documents)>0):
        return "generate"
    else:
        return "direct_answer"

In [119]:
# Generate
def generate(state):
    print_with_time("generate")
    
    question=state["question"]
    documents=state["documents"]
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Please provide a concise answer without repeating the user's question, keeping it under 100 characters. answer solely based on the following context:\n<Documents>\n{context}\n</Documents>.
                """,
            ),
            ("user", "{question}"),
        ]
    )
    
    chain = (
        prompt
        | llm_deepseek_r1
        | StrOutputParser()
    )
    result=chain.invoke({"context": documents, "question": question})
    arr=result.split()
    if len(arr[-1])<=0:
        print_with_time(result)
        raise ValueError("generate出现错误")
    return {"answer": arr[-1] ,"messages": [{"role": "assistant", "content": arr[-1]}]}
    
# s=State(question="手语识别(SLR)技术是什么")
# docs=retrieve(s)
# s=State(question="手语识别(SLR)技术是什么",documents= docs["documents"])
# print(generate(s))

In [120]:
def direct_answer(state):
    print_with_time("direct_answer")
    
    question=state["question"]
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Please provide a concise answer without repeating the user's question, keeping it under 100 characters. 
                """,
            ),
            ("user", "{question}"),
        ]
    )
    
    chain = (
        prompt
        | llm_deepseek_r1
        | StrOutputParser()
    )
    result=chain.invoke({"question": question})
    arr=result.split()
    if len(arr[-1])<=0:
        print_with_time(result)
        raise ValueError("direct_answer出现错误")
    return {"answer": arr[-1] ,"messages": [{"role": "assistant", "content": arr[-1]}]}

# s=State(question="手语识别(SLR)技术是什么")
# print(direct_answer(s))

In [121]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod
from entity import State

graph_builder = StateGraph(State)
graph_builder.add_node("asr", asr)
graph_builder.add_node("tts", tts)
graph_builder.add_node("audio2face", audio2face)
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("grade_documents", grade_documents)
graph_builder.add_node("generate", generate)
graph_builder.add_node("direct_answer", direct_answer)

graph_builder.add_edge(START, "asr")
graph_builder.add_conditional_edges("asr",
                                    route_question,
                                    {"vectorstore": "retrieve", "direct_answer": "direct_answer"}
                                   )
graph_builder.add_edge("retrieve", "grade_documents")
graph_builder.add_conditional_edges("grade_documents", 
                       decide_to_generate,
                       {"generate": "generate", "direct_answer": "direct_answer"}
                      )
graph_builder.add_edge("generate", "tts")
graph_builder.add_edge("direct_answer", "tts")
graph_builder.add_edge("tts", "audio2face")
graph_builder.add_edge("audio2face", END)

checkpointer = MemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)


try:
    display(Image(graph.get_graph().draw_mermaid_png()))
    graph.get_graph().draw_mermaid_png(output_file_path="beauty_assistant.png")
except Exception:
    # This requires some extra dependencies and is optional
    pass


In [122]:
def handle(audio):
    print_with_time("开始处理")

    yield "./video/prologue.mp4","等待响应"
    try:
        
        video=""
        answer=""
        for event in graph.stream({"question_audio":audio}):
            for value in event.values():
                print_with_time("="*100)
                print_with_time(value)
                if value.get("answer_video"):
                    video=value["answer_video"]
                if value.get("answer"):
                    answer=value["answer"]
        if video=="":
            video="./video/prologue.mp4"
        yield video,answer
    except Exception as e:
        error_msg = traceback.format_exc()
        print_with_time(error_msg)
        yield "./video/prologue.mp4",e.args

    print_with_time("处理结束")
# handle("./test.wav")

In [ ]:
import gradio as gr


with gr.Blocks(title="Bolt美女助理") as block:
    gr.HTML(
        f"""
        <h1 style='text-align: center;'>美女助理 </h1>
        <h3 style='text-align: center;'> 让上班的牛马也可以享受美女的服务 </h3>
        <p style='text-align: center;'> Powered by <a href="https://github.com/huggingface/parler-tts"> Bolt</a>
        """
    )
    
    # with gr.Row():
    #     with gr.Column(scale=1):
    #         audio_in = gr.Audio(label="我", sources="microphone",type="filepath")
    #     with gr.Column(scale=4):
    #         with gr.Group():
    #             audio_out = gr.Video(label="美女",autoplay=True,value="./video/prologue.mp4",height=500)
    #             answer = gr.Textbox(show_label=False)
    audio_out = gr.Video(label="美女",autoplay=True,value="./video/prologue.mp4",height=500)
    answer = gr.Textbox(show_label=False)
    audio_in = gr.Audio(label="我", sources="microphone",type="filepath")
    
        
    audio_in.stop_recording(handle, audio_in, [audio_out,answer ])

block.launch(debug=True, share=False, show_api=False, server_port=5000, server_name="0.0.0.0")

* Running on local URL:  http://0.0.0.0:5000

To create a public link, set `share=True` in `launch()`.
